# GOT death analysis in Python with Pandas

In [1]:
import pandas as pd
import numpy as np
import os

## Connect all the db files.

In [2]:
got_deaths = pd.read_csv('/datasets/personalcloud/2021 - GOT Deaths Pracitces/deaths.csv', ';')
got_deaths = got_deaths.loc[:, :'Season']
got_deaths

,Allegiance,Death No.,Episode,Killer,Killers House,Location_id,Weapon_id,Name,Season
0,Night's Watch,1,1,White Walker,None,2,33,Waymar Royce,1
1,Night's Watch,2,1,White Walker,None,2,33,Gared,1
2,Night's Watch,3,1,Ned Stark,House Stark,40,60,Will,1
3,None,4,1,Direwolf,None,40,61,Stag,1
4,None,5,1,Stag,None,40,1,Direwolf,1
...,...,...,...,...,...,...,...,...,...
2219,None,2207,5,Daenerys Targaryen,House Targaryen,15,20,King's Landing Citizen,8
2220,None,2208,5,Daenerys Targaryen,House Targaryen,15,20,King's Landing Citizen,8
2221,None,2209,5,Daenerys Targaryen,House Targaryen,15,20,King's Landing Citizen,8
2222,None,2210,5,Daenerys Targaryen,House Targaryen,15,20,King's Landing Citizen,8


In [3]:
got_weapons = pd.read_csv('/datasets/personalcloud/2021 - GOT Deaths Pracitces/weapon.csv', sep = ';')
got_weapons = got_weapons.loc[:, :'weapon'].dropna()

In [4]:
got_weapons

,id,weapon
0,1,Antler
1,2,Arakh
2,3,Arrow
3,4,Axe
4,5,Barrel
...,...,...
62,63,Tusk
63,64,Unknown
64,65,Water
65,66,Whip


In [5]:
got_location = pd.read_csv('/datasets/personalcloud/2021 - GOT Deaths Pracitces/location.csv', sep = ';')
got_location = got_location.loc[:, :'Location'].dropna()
got_location

,id,Location
0,1,Astapor
1,2,Beyond the Wall
2,3,Braavos
3,4,Casterly Rock
4,5,Castle Black
5,6,Dorne
6,7,Dragonstone
7,8,Dreadfort
8,9,Eastern Road
9,10,Hardhome


## Assign the index and merge the DBs.

In [6]:
# Order the columns to have the index as the first column.

got_deaths = got_deaths.set_index(['Death No.']).sort_index()

In [7]:
# Do a join to bring te names of the weapons and the location to the main table:

got_deaths = pd.merge(got_deaths,got_location, how = 'inner', left_on = 'Location_id', right_on = 'id')
got_deaths = pd.merge(got_deaths,got_weapons, how = 'inner', left_on = 'Weapon_id', right_on = 'id')
got_deaths = got_deaths.drop(columns=['Location_id','Weapon_id', 'id_x', 'id_y'])

# Organize and rename columns:
got_deaths = got_deaths[['Allegiance', 'Episode', 'Killer', 'Killers House', 'Location', 'weapon', 'Name', 'Season']]
got_deaths = got_deaths.rename(columns={'Name':'Dead character', 'weapon':'Weapon'})

In [8]:
# Delete the variable we won't use anymore:

del got_weapons
del got_location

## Data Analysis:

### 1 - ¿How many characters die in GOT?

In [9]:
death_count = got_deaths['Allegiance'].count()
print("The total of death in GOT was: {}.".format(death_count))

The total of death in GOT was: 2224.


### 2 - ¿How many characters die after season 3 in GOT?

In [10]:
death_count_a3s = got_deaths[got_deaths['Season'] > 3]['Allegiance'].count()
print("The # of death in GOT after the 3th season was: {}.".format(death_count_a3s))

The # of death in GOT after the 3th season was: 1950.


### 3 - ¿How many deaths were by Wildfire?

In [11]:
death_count_bwf = got_deaths[got_deaths['Weapon'] == 'Wildfire']['Allegiance'].count()
print("The # of death in GOT by Wildfire was: {}.".format(death_count_bwf))

The # of death in GOT by Wildfire was: 210.


### 4 - ¿How many weapons were used to kill in GOT?

In [12]:
weapons_to_kill =  got_deaths['Weapon'].nunique()
print("The # of weapons used to kill in GOT was: {}.".format(weapons_to_kill))

The # of weapons used to kill in GOT was: 67.


### 5 - ¿Which were the 5 weapons most used to kill in GOT?

In [13]:
top_weapons = pd.DataFrame(got_deaths['Weapon'].value_counts().head(5))
top_weapons['index1'] = top_weapons.index
top_weapons = top_weapons.reset_index(drop = True).drop(columns='Weapon')
top_weapons = top_weapons.values.tolist()
print("The top 5 of weapons most used to kill in GOT were: {}.".format(top_weapons))

The top 5 of weapons most used to kill in GOT were: [['Dragonfire (Dragon)'], ['Sword'], ['Wildfire'], ['Knife'], ['Arrow']].


### 6 - ¿How many episodes with at least one death has GOT?

In [14]:
got_deaths['S-E'] = got_deaths[['Season', 'Episode']].apply(lambda x: ''.join(str(x)), axis=1)
got_deaths = got_deaths.convert_dtypes()
got_deaths['S-E'] = got_deaths['S-E'].apply(lambda x: x.split("\nName",1)[0])
got_deaths['S-E'] = got_deaths['S-E'].str.replace('\n',' ')
got_deaths['S-E'].nunique()
print("Got has in total {} episodes with at least 1 death in it.".format(got_deaths['S-E'].nunique()))
got_deaths = got_deaths.drop(columns = 'S-E')

### 7 - If GOT where a game and kill with and arrow give you 3 points, ¿How many points would you has got during seasons 2, 5 and 6?

In [15]:
points_per_arrow = got_deaths[['Season','Weapon']]
points_per_arrow = points_per_arrow[points_per_arrow['Weapon'] == "Arrow"]
points_per_arrow = points_per_arrow[(points_per_arrow['Season'] == 2) | (points_per_arrow['Season'] == 5) | (points_per_arrow['Season'] == 6)]
points_per_arrow = points_per_arrow.drop(columns = 'Weapon')
points_per_arrow = points_per_arrow['Season'].count() * 3
print("You would have got a total of: {} points.".format(points_per_arrow))

You would have got a total of: 159 points.


### 8 - ¿How many characters would have been killed by House Lannister and House Targaryen if this 2 were just one House called The kill house?

In [16]:
killers_house = pd.DataFrame(got_deaths['Killers House'])
killers_house = pd.DataFrame(killers_house['Killers House'].str.replace('House Lannister', 'The Kill House'))
killers_house = pd.DataFrame(killers_house['Killers House'].str.replace('House Targaryen', 'The Kill House'))
killers_house = killers_house[killers_house['Killers House'] == "The Kill House"]
killers_house = killers_house['Killers House'].count()
print("If the kill house were a real one, this would have killed: {} characters in total.".format(killers_house))

If the kill house were a real one, this would have killed: 1444 characters in total.


### Create a table with the # of deaths by season (axis = 1) and episodes (axis = 0).

In [17]:
pd.options.display.float_format = '{:,.0f}'.format
number_death = got_deaths.copy(deep = True)
number_death = number_death[['Season','Episode']]
number_death['Deaths'] = 1
number_death = number_death.groupby(['Season','Episode']).sum()
number_death = number_death.unstack('Episode')
number_death = number_death.fillna(0)

number_death

Deaths                               
Episode     1  2  3   4   5  6  7  8   9   10
Season                                       
1            7  3  0   1  17  5  5 11   6   3
2            7  1  8   3   4 12 15  0  72   8
3            1  1  4  11   4  7  0  6  45   7
4            5  6 11   1  16  7  7 12  86  19
5            2  8  5  48   2  1  2 23  58   9
6            8  7 10  18   8  0  1  8 133 203
7           54 29 23 205   4  9 10  0   0   0
8            6  1 35   5 844  1  0  0   0   0

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fe744348-fecc-4b7a-803c-44dba24ecb38' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>